<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/master/Equipo_Kung_Fu_Pandas_TP1_7506_Org_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP1 - Real or Not?**

###  **Anexo: Análisis de Modelo de Clasificación**

![Banner_TP1.png](https://drive.google.com/uc?id=1BPA2RF1SDm9bTs1xZfVUa1VQn932E3wr)


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

### Importamos **Librerias**

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

### Levantramos el archivo **train.csv**

In [94]:
df_train = pd.read_csv(r'train.csv', usecols=['keyword','location','text','target']) 
df_train.head()

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Preprocesamiento de datos

In [95]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [96]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7613 non-null   object
 1   location  7613 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [97]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7503 non-null   object
 1   location  7503 non-null   object
 2   text      7503 non-null   object
 3   target    7503 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 293.1+ KB


In [98]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

keyword     False
location    False
text        False
target      False
dtype: bool

### **Análisis & tokenización**

In [0]:
# Función de tokenización
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [100]:
# Creo la serie 'text_vector'
df_train['text_vector']=df_train['text'].apply(tokenize)
df_train.head(2)

,keyword,location,text,target,text_vector
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth..."
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"


In [101]:
# Verifico integridad de DF
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   keyword      7503 non-null   object
 1   location     7503 non-null   object
 2   text         7503 non-null   object
 3   target       7503 non-null   int64 
 4   text_vector  7503 non-null   object
dtypes: int64(1), object(4)
memory usage: 351.7+ KB


In [102]:
# Descargo e importo lista de stopwords para filtrar
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [125]:
# Importo stopwords
from nltk.corpus import stopwords
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [0]:
# Asigno en la variable 'stop' la categoria de stopword por la que voy a realizar el filtro
stop=stopwords.words('english')

In [0]:
def filtro(text_vector):
  text_vector_filtrado = []
  for palabra in text_vector:
    if palabra not in stop:
      text_vector_filtrado.append(palabra)
  return(text_vector_filtrado)

In [106]:
# Tokens filtrados por stopwords en inglés
df_train['text_vector_filtrado']=df_train['text_vector'].apply(filtro)
df_train['text_vector_filtrado'].head()

0    [deeds, reason, earthquake, may, allah, forgiv...
1        [forest, fire, near, la, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
3    [13000, people, receive, wildfires, evacuation...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text_vector_filtrado, dtype: object

In [107]:
# Asigno en una nueva columna el largo del vector 'text_vector'
df_train['elem_vector']=df_train['text_vector'].str.len()
df_train.head(2)

,keyword,location,text,target,text_vector,text_vector_filtrado,elem_vector
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7


In [108]:
# Cuento los elementos del vector filtrado 'text_vector_filtrado'
df_train['elem_vector_filtrado']=df_train['text_vector_filtrado'].str.len()
df_train.head(2)

,keyword,location,text,target,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7


In [109]:
# Verifico estructura del DF
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               7503 non-null   object
 1   location              7503 non-null   object
 2   text                  7503 non-null   object
 3   target                7503 non-null   int64 
 4   text_vector           7503 non-null   object
 5   text_vector_filtrado  7503 non-null   object
 6   elem_vector           7503 non-null   int64 
 7   elem_vector_filtrado  7503 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 527.6+ KB


### Preproceso y categorización de la variable ***Keyword***

In [110]:
# "Limpiamos" la serie de caracteres especiales
df_train['keyword']=df_train['keyword'].str.replace('%20','_')
# Verificamos actualización
df_train['keyword'].unique()

array(['sin keyword', 'ablaze', 'accident', 'aftershock',
       'airplane_accident', 'ambulance', 'annihilated', 'annihilation',
       'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack',
       'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism',
       'blaze', 'blazing', 'bleeding', 'blew_up', 'blight', 'blizzard',
       'blood', 'bloody', 'blown_up', 'body_bag', 'body_bagging',
       'body_bags', 'bomb', 'bombed', 'bombing', 'bridge_collapse',
       'buildings_burning', 'buildings_on_fire', 'burned', 'burning',
       'burning_buildings', 'bush_fires', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'chemical_emergency', 'cliff_fall',
       'collapse', 'collapsed', 'collide', 'collided', 'collision',
       'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone',
       'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge',
       'deluged', 'demolish', 'demolished', 'demolition', 'derail',
       'derailed', 'derailmen

### Generación de **diccionario**

In [117]:
# Listo para generar un diccionario y traducir la serie 'keyword'
keyword_list=df_train['keyword'].unique().tolist()
print(keyword_list)

['sin keyword', 'ablaze', 'accident', 'aftershock', 'airplane_accident', 'ambulance', 'annihilated', 'annihilation', 'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze', 'blazing', 'bleeding', 'blew_up', 'blight', 'blizzard', 'blood', 'bloody', 'blown_up', 'body_bag', 'body_bagging', 'body_bags', 'bomb', 'bombed', 'bombing', 'bridge_collapse', 'buildings_burning', 'buildings_on_fire', 'burned', 'burning', 'burning_buildings', 'bush_fires', 'casualties', 'casualty', 'catastrophe', 'catastrophic', 'chemical_emergency', 'cliff_fall', 'collapse', 'collapsed', 'collide', 'collided', 'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge', 'deluged', 'demolish', 'demolished', 'demolition', 'derail', 'derailed', 'derailment', 'desolate', 'desolation', 'destroy', 'destroyed', 'destruction', 'detonate', 'detonation', 'devastat

In [112]:
#1 Genero lista en espaniol
keyword_list_esp=['sin_keyword','en llamas','accidente','replica','accidente_avion','ambulancia','aniquilado','aniquilacion','apocalipse','armageddon','ejercito','incendio_intencional','piromano','ataque','atacado','avalancha','combate/batalla/lucha','bioterror','bioterrorismo','resplandecer/arder','resplandeciente/ardiente','hemorragia','exploto','plaga','ventisca','sangre','sangriento','estallido','bolsa_de_cadaver','embolsado_de_cadaver','bolsas_de_cadaver','bomba','bombardeado','bombardeo','colapso_de_puente','incendio_de_edificios','edificios_en_llamas','quemado','quemando/incendiando','incendio_edificios','incendio_de_matorrales/arbustos','bajas/muertes/perdidas','baja/muerte/perdida','catastrofe','catastrofico','emergencia_quimica','caida_acantilado','colapso','colapsado','colisionar','colisiono','colision','choque','estrellado','aplastamiento','aplastado','toque_de_queda','ciclon','da¤o','peligro','muerto','muerte','muertes','escombros','diluvio','inundado','demoler','demolido','demolicion','descarrilar','descarrilado','descarrilamiento','desolado','desolacion','destruir','destruido','destruccion','detonado','detonacion','devastado','devastacion','desastre','dislacado/desplazado','sequia','ahogar','ahogado','ahogamiento/ahogandose','tormenta_polvo','terremoto','electrocutar','electrocutado','emergencia','plan_de_emergencia','servicio_de_emergencia','engullido','epicentro','evacuar','evacuado','evacuacion','explotar','exploto/detonado/estallado','explosion','testigo_ocular','hambruna','fatal','fatalidades','fatalidades','miedo','fuego','camion_bombero','primeros_en_responder','llamas','aplastado','inundacion','inundaciones','inundaciones','incendio_forestal','incendio_forestales','granizo','tormenta_granizo','da¤o','peligro','peligroso','ola_de_calor','infierno','secuestro','secuestrador','secuestro','rehen','rehenes','huracan','herido/lesionado','heridas/lesiones','herida/lesion','inundado','inundacion','deslisamiento_tierra','lava','rayo','golpe_fuerte/estampido_fuerte','asesinato_en_serie','asesino_en_serie','masacre','caos','fusion','militar','alud_de_lodo','desastre_natural','desastre_nuclear','reactor_nuclear','obliterar/destruir/eliminar','borrado/destruido/eliminado','obliteracion/destruccion/eliminacion','derrame_de_petroleo','brote','pandemonio/confusion/caos','panico','tener_panico','polic¡a','cuarentena','en_cuarentena','emergencia_radiacion','tormenta','arrasado','refugiados','rescate','rescatado','rescatadores','disturbio','disturbios','escombros','ruina','tormenta_de_arena','grito','gritando','gritos','sismico','sumidero/pozo','hundiendose','sirena','sirenas','humo','tormenta_de_nieve','tormenta','camilla','fallo_estructural','bomba_suicida','suicida','bombardeo_suicida','hundido','sobrevivir','sobrevivio','sobreviviente','terrorismo','terrorista','amenaza','trueno','tormenta_electrica','tornado','tragedia','atrapado','trauma','traumatizado','problema','tsunami','tornado/torbellino','tifon','transtorno','tormenta_violenta','volcan','zona_de_guerra','arma','armas','torbellino','incendios_forestales','incendio_forestal','tormenta_de_viento','moretoneado','moretones','naufragio','restos','naufrago']
print(keyword_list_esp)

['sin_keyword', 'en llamas', 'accidente', 'replica', 'accidente_avion', 'ambulancia', 'aniquilado', 'aniquilacion', 'apocalipse', 'armageddon', 'ejercito', 'incendio_intencional', 'piromano', 'ataque', 'atacado', 'avalancha', 'combate/batalla/lucha', 'bioterror', 'bioterrorismo', 'resplandecer/arder', 'resplandeciente/ardiente', 'hemorragia', 'exploto', 'plaga', 'ventisca', 'sangre', 'sangriento', 'estallido', 'bolsa_de_cadaver', 'embolsado_de_cadaver', 'bolsas_de_cadaver', 'bomba', 'bombardeado', 'bombardeo', 'colapso_de_puente', 'incendio_de_edificios', 'edificios_en_llamas', 'quemado', 'quemando/incendiando', 'incendio_edificios', 'incendio_de_matorrales/arbustos', 'bajas/muertes/perdidas', 'baja/muerte/perdida', 'catastrofe', 'catastrofico', 'emergencia_quimica', 'caida_acantilado', 'colapso', 'colapsado', 'colisionar', 'colisiono', 'colision', 'choque', 'estrellado', 'aplastamiento', 'aplastado', 'toque_de_queda', 'ciclon', 'da¤o', 'peligro', 'muerto', 'muerte', 'muertes', 'escomb

In [118]:
#1 Genero el diccionario eng-esp de la serie 'Keyword'
keyword_dicc={}
y=0
for x in keyword_list:
  keyword_dicc[x]=keyword_list_esp[y]
  y=y+1
  #print(keyword_dicc)

print(keyword_dicc)

{'sin keyword': 'sin_keyword', 'ablaze': 'en llamas', 'accident': 'accidente', 'aftershock': 'replica', 'airplane_accident': 'accidente_avion', 'ambulance': 'ambulancia', 'annihilated': 'aniquilado', 'annihilation': 'aniquilacion', 'apocalypse': 'apocalipse', 'armageddon': 'armageddon', 'army': 'ejercito', 'arson': 'incendio_intencional', 'arsonist': 'piromano', 'attack': 'ataque', 'attacked': 'atacado', 'avalanche': 'avalancha', 'battle': 'combate/batalla/lucha', 'bioterror': 'bioterror', 'bioterrorism': 'bioterrorismo', 'blaze': 'resplandecer/arder', 'blazing': 'resplandeciente/ardiente', 'bleeding': 'hemorragia', 'blew_up': 'exploto', 'blight': 'plaga', 'blizzard': 'ventisca', 'blood': 'sangre', 'bloody': 'sangriento', 'blown_up': 'estallido', 'body_bag': 'bolsa_de_cadaver', 'body_bagging': 'embolsado_de_cadaver', 'body_bags': 'bolsas_de_cadaver', 'bomb': 'bomba', 'bombed': 'bombardeado', 'bombing': 'bombardeo', 'bridge_collapse': 'colapso_de_puente', 'buildings_burning': 'incendio_

In [114]:
# Mapeo los valores de las key del diccionario keyword_dicc
df_train['keyword_esp']=df_train['keyword'].map(keyword_dicc)
df_train.head(2)

,keyword,location,text,target,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,keyword_esp
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,sin_keyword
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,sin_keyword


In [115]:
# Verifico el top 20 de las categorias mas frecuentes para la serie keyword 
df_train['keyword_esp'].value_counts().sort_values(ascending=False).head(20)

da¤o                  82
fatalidades           82
inundaciones          72
peligro               70
tormenta              69
aplastado             65
escombros             65
secuestro             64
inundado              62
sin_keyword           56
incendio_forestal     52
inundacion            45
diluvio               42
armageddon            42
bolsas_de_cadaver     41
tornado/torbellino    40
tormenta_de_viento    40
miedo                 40
sirena                40
evacuar               40
Name: keyword_esp, dtype: int64

### → **Scikit Learn**: Ejecución de **One Hot Encoding** para clasificar

In [119]:
# Dummifico las categorías
df_train_dummies=pd.get_dummies(df_train.keyword)
df_train_dummies.head(2)

,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,bombed,bombing,bridge_collapse,buildings_burning,buildings_on_fire,burned,burning,burning_buildings,bush_fires,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:
# Hago un join con df_train y df_train_dummies
df_train_merged = pd.concat([df_train,df_train_dummies],axis='columns')
df_train_merged.head(2)

,keyword,location,text,target,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,keyword_esp,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,sin_keyword,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,sin_keyword,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [121]:
# Elimino las columnas 'keyword_esp' y 'sin keyword'
df_train_merged_final=df_train_merged.drop(['keyword_esp','sin keyword'],axis='columns')
df_train_merged_final.head(2)

,keyword,location,text,target,text_vector,text_vector_filtrado,elem_vector,elem_vector_filtrado,ablaze,accident,aftershock,airplane_accident,ambulance,annihilated,annihilation,apocalypse,armageddon,army,arson,arsonist,attack,attacked,avalanche,battle,bioterror,bioterrorism,blaze,blazing,bleeding,blew_up,blight,blizzard,blood,bloody,blown_up,body_bag,body_bagging,body_bags,bomb,bombed,...,snowstorm,storm,stretcher,structural_failure,suicide_bomb,suicide_bomber,suicide_bombing,sunk,survive,survived,survivors,terrorism,terrorist,threat,thunder,thunderstorm,tornado,tragedy,trapped,trauma,traumatised,trouble,tsunami,twister,typhoon,upheaval,violent_storm,volcano,war_zone,weapon,weapons,whirlwind,wild_fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,sin keyword,sin location,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",13,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,sin keyword,sin location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Exporto a .csv para verificar estructura
df_train_merged_final.to_csv('columna_texto_vector_filtrado.csv')

In [123]:
print(df_train_merged_final['text_vector_filtrado'])

0       [deeds, reason, earthquake, may, allah, forgiv...
1           [forest, fire, near, la, ronge, sask, canada]
2       [residents, asked, shelter, place, notified, o...
3       [13000, people, receive, wildfires, evacuation...
4       [got, sent, photo, ruby, alaska, smoke, wildfi...
                              ...                        
7604    [worldnews, fallen, powerlines, glink, tram, u...
7605    [flip, side, im, walmart, bomb, everyone, evac...
7606    [suicide, bomber, kills, 15, saudi, security, ...
7608    [two, giant, cranes, holding, bridge, collapse...
7612    [latest, homes, razed, northern, california, w...
Name: text_vector_filtrado, Length: 7503, dtype: object


Vectorizo con el feature **CountVectorizer**

In [126]:
# Defino los documentos/vector (para fragmentar la cadena de palabras)
documents=(df_train_merged_final['text'])
documents

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7604    #WorldNews Fallen powerlines on G:link tram: U...
7605    on the flip side I'm at Walmart and there is a...
7606    Suicide bomber kills 15 in Saudi security site...
7608    Two giant cranes holding a bridge collapse int...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7503, dtype: object

In [0]:
# Se ignorara todas las palabras que se encuentren en la lista stop_words
vectorizer=CountVectorizer(stop_words='english')
#Ajustamos el conjunto de datos del documento al objeto countvectorizer
vectors=vectorizer.fit_transform(documents)
# conseguimos lal ista de palabras que han sido clasificadas
names=vectorizer.get_feature_names()
#names

In [128]:
# creamos una matriz. El valor correspondiente a la (fila, columna) sera la frecuencia de esa palabra
doc_array=vectorizer.transform(documents).toarray()
doc_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Implementación de **BoW** O **Bag of Word**

In [129]:
# convertimos la matriz en una estructura de datos
frecuency_matrix=pd.DataFrame(data=doc_array, columns=names)
frecuency_matrix

,00,000,0000,007npen6lg,00cy9vxeff,00end,00pm,01,02,0215,02elqlopfk,02pm,03,030,033,034,039,03l7nwqdje,04,05,05th,06,060,061,06jst,07,073izwx0lb,08,0840728,0853,087809233445,0880,08lngclzsj,09,0abgfglh7x,0ajisa5531,0blkwcupzq,0btniwagt1,0bvk5tub4j,0c1y8g7e9p,...,ûªre,ûªs,ûªt,ûªve,ûï,ûïa,ûïafter,ûïairplane,ûïall,ûïcat,ûïdetonate,ûïfor,ûïhatchet,ûïi,ûïlittle,ûïlove,ûïmake,ûïnews,ûïnobody,ûïnumbers,ûïparties,ûïplans,ûïrichmond,ûïsippin,ûïstretcher,ûïthat,ûïthe,ûïwe,ûïwhen,ûïyou,ûò,ûò800000,ûòthe,ûòåêcnbc,ûó,ûóher,ûókody,ûónegligence,ûótech,ûówe
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7499,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7501,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Set de datos de **Pruebas** y de **Entrenamiento**

In [0]:
# Divido los conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test= train_test_split(df_train_merged_final['text'],\
                                                   df_train_merged_final['target'], random_state=1)

In [131]:
print('Cant total de filas en el set de datos: {}'.format(df_train_merged_final.shape[0]))

Cant total de filas en el set de datos: 7503


In [132]:
print('Cant de filas en el set de datos de entrenamiento: {}'.format(X_train.shape[0]))

Cant de filas en el set de datos de entrenamiento: 5627


In [133]:
print('Cant de filas en el set de datos de pruebas: {}'.format(X_test.shape[0]))

Cant de filas en el set de datos de pruebas: 1876


### Ejecución de BoW para procesar los datos de pruebas

In [0]:
# Iniciamos el metodo countvectorizer otra vez
count_vector=CountVectorizer()

In [135]:
# Ajustamos el set de entrenamiento
training_data=count_vector.fit_transform(X_train)
training_data

<5627x17768 sparse matrix of type '<class 'numpy.int64'>'
	with 81986 stored elements in Compressed Sparse Row format>

In [136]:
# Ajustamos el set de pruebas
testing_data=count_vector.transform(X_test)
testing_data

<1876x17768 sparse matrix of type '<class 'numpy.int64'>'
	with 23763 stored elements in Compressed Sparse Row format>

### Implementación de Naives Bayes Multinomial para clasificar

In [137]:
# Iniciamos los metodos y ajustamos los datos en el clasificador
naives_bayes=MultinomialNB()
naives_bayes.fit(training_data, y_train)
naives_bayes.fit(testing_data, y_test)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
# Realizamos predicción con el conjunto de pruebas
predicciones=naives_bayes.predict(testing_data)

### Evaluación del modelo

Para todas las mediciones - cuyo rango de probabilidad es entre o y 1 - tener una puntuación cercana al valor 1 es buen estimador del comportamiento del modelo


In [144]:
# El accuracy informa cómo el clasificador realiza la predicción correcta de tweets
print("Accuracy: ",format(accuracy_score(y_test,predicciones)))

Accuracy:  0.8960554371002132


In [141]:
# La precisión informa la proporción de tweets que clasificamos como verdaderos
print("Métrica de Precisión del Modelo: ",format(precision_score(y_test,predicciones)))

Métrica de Precisión del Modelo:  0.9457013574660633


In [145]:
# La confianza o recall, da la proporción que realmente era verdadero y fueron clasificados como tal.
print("Confianza o recall del Modelo: ",format(f1_score(y_test, predicciones)))

Confianza o recall del Modelo:  0.8654244306418221
